## Pricing the term structure with linear regressions

Tobias Adriann, Richard K. Crump, Emanuel Moench

#### Notebook Author: Pedro Coelho


### Introdução

Até o começo dos anos 1990, acreditava-se que a estrutura a termo da taxa de juros era apenas a realização das expectativas do mercado para o caminho da taxa de juros futuros. Porém dos anos 1990 em diante começou a ficar claro que os investidores exigiam um prêmio de risco para investir em títulos de prazo mais longos e quanto mais longo esse prazo, maior o prêmio. Desde então começaram a surgir modelos para estimar esse prêmio de risco não observado, como os modelos de Kim e Wright (2005) e Cochrane e Piazzesi (2005, 2008).

In [ ]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA
from pathlib import Path

pd.options.mode.chained_assignment = None


path = str(Path().resolve())


def plot_interest_rate_on_time(
    curve_df: pd.DataFrame | pd.Series,
    title: str,
    y_tickformat: str = ".2%",
    rangemode: str = "tozero",
    y_dtick: float = 0.02,
    template: str = "simple_white",
    xaxis_title: str = "Dates",
    yaxis_title: str = "Interest Rate",
    use_area_chart: bool = False,
    showlegend: bool = True,
):
    if use_area_chart:
        fig = px.area(curve_df, title=title, template=template)
    else:
        fig = px.line(curve_df, title=title, template=template)
    return (
        fig.update_yaxes(
            tickformat=y_tickformat,
            rangemode=rangemode,
            dtick=y_dtick,
            showgrid=True,
        )
        .update_xaxes(
            tickformat="%b %Y",
            ticklabelmode="period",
            showgrid=True,
        )
        .update_layout(
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            showlegend=showlegend,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
                title="",
            ),
        )
    )


def plot_interest_rate_curve(
    curve_df: pd.DataFrame | pd.Series,
    title: str,
    y_tickformat: str = ".2%",
    rangemode: str = "tozero",
    y_dtick: float = 0.01,
    template: str = "simple_white",
    xaxis_title: str = "Years",
    yaxis_title: str = "Interest Rate",
    use_area_chart: bool = False,
    showlegend: bool = True,
):
    if use_area_chart:
        fig = px.area(curve_df, title=title, template=template)
    else:
        fig = px.line(curve_df, title=title, template=template)
    return (
        fig.update_yaxes(
            tickformat=y_tickformat,
            rangemode=rangemode,
            dtick=y_dtick,
            showgrid=True,
        )
        .update_xaxes(
            rangemode="tozero",
            dtick=1,
            showgrid=True,
        )
        .update_layout(
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            showlegend=showlegend,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
                title="",
            ),
        )
    )

### Modelos afins de taxas de juros 

Geralmente partem de 3 hipóteses:

1. A taxa estocástica de desconto é exponencialmente afim em relação aos choques econômicos;
2. O preço do risco é afim em relação as variáveis do modelo;
3. E os erros/inovações do modelo e o log das taxas de juros são condicionalmente gaussianas.

Empiricamente eles são estimados através de métodos de máxima verossimilhança que são capazes de explorar as hipóteses de distribuição e as restrições de não arbitragem.

O Modelo ACM também é um modelo afim para estimar a estrutura a termo da taxa de juros, com a diferença que é estimado através de Minimos Quadrados Ordinários. A grande diferença entre os modelos de máxima verossimilhança e o modelo de ACM é a hipótese sobre autocorrelação dos erros. O primeiro assume que não há autocorrelação dos erros o que leva a perda de previsibilidade dos retornos que não é capturado pelos fatores de apreçamento, já o modelo de ACM, que é baseado em MQO, não há necessidade de se fazer nenhuma hipótese sobre autocorrelação dos erros.

### Hipóteses do modelo de ACM

1. Primeiramente eles assumem que as variáveis ${X}_{t}$ de tamanho K x 1 evoluem de acordo com um modelo de Vetores Autorregressivos:

    ${X}_{t+1} = {\mu} + {\Phi}{X}_{t} + {\nu}_{t+1}$

2. Dada as condições de não-arbitragem de um modelo de taxa de juros deve existir uma taxa estocástica de desconto ${M}_{t}$ tal que:

    ${P}_{t}^{(n)} = {E}_{t}[{M}_{t+1}{P}_{t+1}^{(n-1)}]$

    sendo que a taxa estocástica de desconto é exponencialmente afim em relação aos choques da economia

    ${M}_{t+1} = exp({-}{r}_{t} -\frac{1}{2}{\lambda}_{t}'{\lambda}_{t} - {\lambda}_{t}'{\Sigma}^{-\frac{1}{2}}{\nu}_{t+1})$

    e ${r}_{t} = ln {P}_{t}$ é a taxa livre de risco continuamente composta.
    
3. Eles também assumem que o preço do risco é afim em relação as variáveis do modelo como o sugerido por Duffee (2002):

    ${\lambda}_{t} = {\Sigma}^{-\frac{1}{2}}({\lambda}_{0} + {\lambda}_{1}{X}_{t})$

4. E assumem que os erros observados ${\nu}_{t+1}$ são condicionalmente gaussianas

    ${\nu}_{t+1}|{({X}_{s})}_{s=0}^{t} \sim {N(0, \Sigma)}$


Sendo ${rx}_{t+1}^{(n-1)}$ o log do excesso de retorno de um título com vencimento em n períodos tal que:

${rx}_{t+1}^{(n-1)} = ln({P}_{t+1}^{(n-1)}) - ln({P}_{t}^{(n)}) - {r}_{t}$

Com um pouco de álgebra e utilizando as hipóteses iniciais chegamos a seguinte equação:

${rx}_{t+1}^{(n-1)} = {\beta}^{(n-1)'}({\lambda}_{0} + {\lambda}_{1}{X}_{t}) -\frac{1}{2}({\beta}^{(n-1)'}{\Sigma}{\beta}^{(n-1)} + {\sigma}^{2}) +{\beta}^{(n-1)'}{\nu}_{t+1} + {e}_{t+1}^{(n-1)}$

Assumindo que ${e}_{t+1}^{(n-1)}$ é independente e identicamente distribuído com variância ${\sigma}^{2}$

Quebrando a equação acima, o retorno em excesso de um título com vencimento em n em t+1 é igual a:

* ${\beta}^{(n-1)'}({\lambda}_{0} + {\lambda}_{1}{X}_{t})$: o retorno esperado do título com vencimento em n no período t

* $-\frac{1}{2}({\beta}^{(n-1)'}{\Sigma}{\beta}^{(n-1)} + {\sigma}^{2})$: menos um ajuste de convexidade, necessário em modelos lineares de taxa de juros

* ${\beta}^{(n-1)'}{\nu}_{t+1}$: mais o retorno adicional ímplicito no preço do título 

* ${e}_{t+1}^{(n-1)}$: e o erro de apreçamento.

Ao empilhar a equação acima para todos os vencimentos e para todos os períodos de tempo obtemos:

$rx = {\beta}({\lambda}_{0}{i}_{T}' + {\lambda}_{1}{X\_}) -\frac{1}{2}({B}^{*}vec{\Sigma} + {\sigma}^{2}{i}_{N}){i}_{T}' + {\beta'}V + E$




### A estimação do modelo ACM é dividida em 3 etapas:

1. Primeiro os fatores do modelo são decompostos em componentes previsíveis e fatores de inovação através 
da regressão dos fatores em relação aos mesmos fatores defasados. 

2. Na segunda etapa é estimada a exposição dos retornos em excesso em relação aos fatores defasados e em relação 
aos fatores de inovação contemporâneos.

3. Finalmente é obtido os preços de mercado do risco atráves de uma regressão *cross-sectional* das exposições dos retornos em relação aos fatores de apreçamento defasados e os fatores de inovações contemporâneos.

A grande vantagem do modelo de ACM é que ele é estimado através de um MQO que é computacionalmente muito mais rápido que os modelos de verossimilhança que não possuem fórmulas fechadas e exigem soluções numéricas mais complexas.

### Começamos extraindo a curva de juros de DI1 desde janeiro de 2011 até abril de 2021

Neste caso a curva foi interpolada previamente para todos os dias úteis do período e para vértices diários de até 10 anos. 

A curva DI1, apesar de ser uma curva de futuros, é uma excelente proxy da taxa zero brasileira por ser mais líquida e possuir mais vértices que a curva de títulos públicos brasileiros.

In [ ]:
base_count = 12

first_date = pd.to_datetime("2011-01-01")
last_date = pd.to_datetime("2021-04-30")

# uploading the curve date
curve = pd.read_csv(f"{path}/data/di1_monotonic_curve.csv", index_col=0)
# making sure the data has the correct data type
curve = curve.astype(float)
# converting the columns names to monthly
curve.columns = [int(column) for column in curve.columns]
# converting the date index to a datetime index
curve.index = pd.to_datetime(curve.index)
# filtering the data until the last date
curve = curve[curve.index <= last_date]
curve = curve[curve.index > first_date]

print(f"Min Date: {curve.index.min():%Y-%b-%d}, Max Date: {curve.index.max():%Y-%b-%d}")

Extraída a curva original nos filtramos a curva para uma periodicidade mensal, usando o último dia útil de cada mês e também filtramos os vértices da curva para que também sejam somente mensais. É importante que os vértices tenham a mesma periodicidade para facilitar no cálculo do excesso de retorno da curva.

In [ ]:
monthly_curve = curve.resample("BME").last()
fig_original_curve = plot_interest_rate_on_time(
    monthly_curve.loc[:, 60],
    title=f"{str(int(60 / 12))} Year Yield",
    y_dtick=None,
    showlegend=False,
)
fig_original_curve.show()

Com a curva mensal podemos calcular o excesso de retorno:

${rx}_{t+1}^{(n-1)} = ln({P}_{t+1}^{(n-1)}) - ln({P}_{t}^{(n)}) - {r}_{t}$

onde ${r}_{t}$ é a taxa livre de risco no instante *t*

Aqui nos conseguimos observar a importância dos vértices e das datas terem a mesma periodicidade. Também é importante notar que nossa taxa livre de risco também segue a periodicidade da curva, se a curva fosse diária a taxa livre de risco seria o CDI, como nesse caso ela á mensal consideramos o primeiro mês de vencimento como a taxa livre de risco.

In [ ]:
def calculate_excess_returns(df: pd.DataFrame, base_count: int) -> pd.DataFrame:
    """Calculate the yield curve excess returns.

    Args:
        df (pd.DataFrame): The yield curve time series.
        base_count (int): The base count of the yield curve

    Returns:
        pd.DataFrame: _description_
    """
    excess_returns = pd.DataFrame(
        index=df.index,
        columns=df.columns,
        dtype="float64",
    )
    # short-term risk-free rate is the shortest rate
    r_t = df.iloc[:, 0].shift(1) * (1 / base_count)
    # skipping the first column which is the short-term rate
    for n in df.columns[1:]:
        # bond price with maturity n-1 at t
        p_n_minus_1_d = -df.loc[:, n - 1] * ((n - 1) / base_count)
        # bond price with maturity n at t - 1
        p_n_d_minus_1 = (-df.loc[:, n] * (n / base_count)).shift(1)
        # Excess return
        excess_returns.loc[:, n] = p_n_minus_1_d - p_n_d_minus_1 - r_t

    return excess_returns

Convertemos a curva de juros para taxas contínuas e calculamos o excesso de retorno.

Importante ressaltar que no calculo do excesso de retorno nos perdemos a primeira data de observação e portanto a curva é reindexada excluindo a primeira data.

In [ ]:
# converting the curve data to continuous compounding
curve_exp = np.log(1 + monthly_curve)
# calculating the excess return
excess_returns = calculate_excess_returns(curve_exp, base_count)
# dropping the first row of data
excess_returns = excess_returns.dropna(how="all", axis=0).dropna(how="all", axis=1)
# reindexing the base curve
curve_exp = curve_exp.loc[excess_returns.index, :]

# Setup
# just storing the number of tenor on the curve
n_tenors = excess_returns.shape[1]
# storing the tenor in a list for latter use
tenors = excess_returns.columns
# getting the sample size for later use
sample_size = curve_exp.shape[0] - 1

Calculado os excessos de retornos agora usamos a curva exponencial para calcular os PCs.

PCA é uma técnica utilizada em finanças para reduzir os fatores de modelos multifatoriais de modo a torná-los mais tratáveis. Inicialmente Scheinkman and Litterman (1991) argumentavam que somente 3 fatores são suficientes para explicar todo o comportamento da curva de juros, porém em estudos mais recentes, como os de Cochrane and Piazzesi (2005, 2008) e Duffe (2011) já mostram que são necessários mais fatores para explicar os retornos dos títulos de dívida pública americanos.

Utilizamos 5 fatores como o especificado no artigo do ACM.

No caso da estrutura a termo de taxa de juros a literatura já foi capaz de identificar quem são os 4 primeiros PCs, sendo eles nível, inclinação, curvatura e torção. ACM nos mostra que o 5º PC também é relevante apesar de não haver uma interpretação clara sobre o que ele representa na curva de juros.

Abaixo nos calculamos os PCs e mostramos no tempo o comportamento dos fatores. Os fatores estão padronizados.

In [ ]:
# Step 0 - get the PCA factor series
# the ACM paper show us that 5 factors are needed,
n_factors = 5
# instantiating the PCA class
pca = PCA(n_components=n_factors)

# Naming the columns, this will be useful later
pca_columns = ["PC" + str(i) for i in range(1, n_factors + 1)]

# Calculating the PCs from the curve, we already get the demeaned and standardized data from the PCA
transformed_values = pca.fit_transform(curve_exp.values)

# creating a DataFrame with the PCAs for better use
pca_factors = pd.DataFrame(
    data=transformed_values, index=curve_exp.index, columns=pca_columns
)

# Plotting the PCs on a time axis
fig_pca = plot_interest_rate_on_time(
    pca_factors,
    title="PCs no tempo",
    y_tickformat=".2f",
    yaxis_title="PC Value",
    y_dtick=None,
)
fig_pca.show()

Também é interessante observar quanto cada PC explica da variância total dos fatores. 

Nesse caso observamos que nível e inclinação (PC1 e PC2) já são responsáveis por mais de 99% da variância observada. 
Apesar do PC4 e PC5 apresentarem menos de 2 bps da variância da curva não há prejuízo computacional em mantê-los no cálculo e assim continuamos seguindo a especificação proposta no artigo de ACM.

In [ ]:
# Plotting the explained Variance for each PC
bar_chart = px.bar(
    x=pca_columns,
    y=pca.explained_variance_ratio_,
    template="simple_white",
    text=pca.explained_variance_ratio_,
)
bar_chart.update_yaxes(tickformat=".2%")
bar_chart.update_layout(
    showlegend=False, xaxis_title="PCs", yaxis_title="Pct of Explained Variance"
)
bar_chart.update_traces(textposition="outside", texttemplate="%{text:.3%}")
bar_chart.show()

## Estimando o modelo de ACM

### Primeiro passo, estimar os parâmetros do nosso Vetor Autoregressivo

Calculados os PCs usamos eles para estimar nosso modelo de Vetor Autoregressivo:

${X}_{t+1} = {\mu} + {\Phi}{X}_{t} + {\nu}_{t+1}$

E lembrando que os assumimos que os choques ${\nu}_{t+1}$ são condicionalmente Gaussianas com matriz de var-covar ${\Sigma}$:

${\nu}_{t+1}|{({X}_{s})}_{s=0}^{t} \sim {N(0, \Sigma)}$


Por MQO temos que $\hat{\beta} = ({X}{\_}'{X}{\_})^{-1}{X}{\_}'{X}$ onde ${\mu}$ é igual a primeira coluna (as constantes da regressão) de $\hat{\beta}$ e ${\Phi}$ da segunda coluna em diante.

Calculados os resíduos da regressão (as inovações) $\hat{\nu}_{t+1}$, empilhamos os valores na matriz $\hat{V}$ e usamos para calcular um estimador da matriz de covariância das variáveis do modelo:

$\hat{\Sigma} = \hat{V}\hat{V}' / T$

In [ ]:
# Step 1 - VAR for the PCA equities


def var_one(
    Y: npt.NDArray[np.float64] | pd.DataFrame | pd.Series,
    X: npt.NDArray[np.float64] | pd.DataFrame | pd.Series,
    n_factors: int,
) -> tuple[npt.NDArray[np.float64], npt.NDArray[np.float64], npt.NDArray[np.float64]]:
    # The VAR(1) estimator is given by equation (3.2.10) from Lutkepohl's book.
    mat_X = np.matrix(X)
    mat_Y = np.matrix(Y).T
    beta_hat = mat_Y @ (mat_X.T @ np.linalg.inv(mat_X @ mat_X.T))
    # Residuals matrix
    res = mat_Y - (beta_hat @ mat_X)
    # unbiased estimate of the covariance matrix
    unbiased_cov_estimate = (res @ res.T) / (mat_X.shape[1] - n_factors - 1)
    return beta_hat, res, unbiased_cov_estimate


# dropping the first observations of the PCs, X_t+1
X_t_plus_1 = pca_factors.iloc[1:].copy()
# dropping the last observations of the PCs, X_t
X_t = pca_factors.iloc[:-1].copy()
# adding constant
X_t["const"] = 1
# making sure that the constant is the first column and the CPs are in order
X_t = X_t[["const"] + pca_columns].T

B_hat, V_hat, S_hat = var_one(X_t_plus_1, X_t, n_factors)

# Computes matrices Mu and Phi of the VAR(1) of the paper.
# Equation 1 of ACM paper
Mu_hat = B_hat[:, 0]
Phi_hat = B_hat[:, 1 : n_factors + 1]


### Segundo passo, regredimos o excesso de retorno em relação aos fatores defasados e o fatores contemporâneos de inovação.

$rx = a{i}_{T}' + {\beta}'\hat{V} + c{X\_} + E$

Juntando os regressores em uma matriz $(2K + 1)\times T$, $\widetilde{Z} = [{i}_{T}\hat{V}'{X\_}']'$
os estimadores são obtidos através da seguinte equação:

$[\hat{a}\hat{\beta}'\hat{c}] = rx\widetilde{Z}'(\widetilde{Z}\widetilde{Z}')^{-1}$

Coletamos os erros da regressão na matriz $\hat{E}$ e obtemos o estimador $\hat{\sigma}^{2} = tr(\hat{E}\hat{E}') / NT$. tr é o traço de uma matriz quadrada, que é a soma de todos os valores da diagonal principal da matriz.

In [ ]:
# Step 2 - Excess return equation

# dropping the first line
mat_rx = excess_returns.iloc[1:].values.T.astype(float)

# Collecting the regressors into the (2K + 1)xT matrix
Z_stack = np.concatenate(
    (
        np.ones((1, sample_size)),
        V_hat,
        pca_factors.iloc[:-1].values.T,
    )
)

# Equation 15 of ACM
D_hat = mat_rx @ (Z_stack.T @ np.linalg.inv(Z_stack @ Z_stack.T))
a_hat = D_hat[:, 0]
b_hat = D_hat[:, 1 : n_factors + 1].T
c_hat = D_hat[:, n_factors + 1 :]

# Residuals of the regression
E_hat = mat_rx - D_hat @ Z_stack
# We then estimate sigma^2 = trace(E_hat x E_hat.T) / (number_of_tenors * sample_size)
sigma2_hat = np.trace(E_hat @ E_hat.T) / (n_tenors * sample_size)


${B}^{*} = [vec({\beta}^{(1)}{\beta}^{(1)'}) ... vec({\beta}^{(N)}{\beta}^{(N)'})]'$ que é uma matriz com tamanho ${N} x {K}^{2}$

vec é a vetorização de uma matriz, onde empilhamos as colunas de uma matriz de tamanho *m x n* em um vetor coluna de tamanho *mn x 1*

In [ ]:
# Builds the estimate of the B* matrix, defined in equation (13) of the paper
new_shape = (1, n_factors**2)
B_star_hat = np.zeros((n_tenors, n_factors**2))
for i in range(0, n_tenors):
    B_star_hat[i, :] = np.reshape(b_hat[:, i] @ b_hat[:, i].T, new_shape)

### Terceiro passo, estimamos o preço do risco ${\lambda}_{0}$ e ${\lambda}_{1}$ através de uma regressão transversal:

$\hat{\lambda}_{0} = (\hat{\beta}\hat{\beta}')^{-1}\hat{\beta}(\hat{a} + \frac{1}{2}(\hat{B}^{*}vec(\hat{\Sigma}) + \hat{\sigma}_{tN}))$

$\hat{\lambda}_{1} = (\hat{\beta}\hat{\beta}')^{-1}\hat{\beta}\hat{c}$

Vale notar que tanto $\hat{\lambda}_{0}$ quanto $\hat{\lambda}_{1}$ são constantes, portanto o seu valor está diretamente relacionado ao período amostral selecionado.

In [ ]:
# Step 3 - Estimate price of risk parameters

# vec(Sigma_hat)
vec_s_hat = np.reshape(S_hat, (n_factors**2, 1))
# B_start * vec(Sigma_hat)
b_start_vec_sigma_hat = np.dot(B_star_hat, np.reshape(S_hat, (n_factors**2, 1)))
# sigma2_hat * (vector of ones with n tenors size)
sigma_2_hat_n = sigma2_hat * np.ones((n_tenors, 1))
# calculating the matrix product inv(beta_hat x beta_hat_T) x beta_hat
bb_inv_b = np.linalg.inv(b_hat @ b_hat.T) @ b_hat

l_zero_hat = bb_inv_b @ (a_hat + 0.5 * (B_star_hat @ vec_s_hat + sigma_2_hat_n))
l_one_hat = np.dot(bb_inv_b, c_hat)

### Estimando a curva de juros

Com os valores de $({\Phi}, {\Sigma}, {\sigma}, {\beta}, {\lambda}_{0}, {\lambda}_{1})$ devidamente estimados podemos calcular os preços dos títulos de cupom zero para a nossa amostra.

Lembrando que dada as hipóteses feitas inicialmente o preço de um título de cupom zero é exponencialmente afim em relação as variáveis do modelo:

$ln{P}_{t}^{(n)} = {A}_{n} + {B}_{n}'{X}_{t} + {\mu}_{t}^{(n)}$

Substituindo a equação acima na fórmula de excesso de retorno temos que:

${rx}_{t+1}^{(n-1)} = {A}_{n-1} + {B}_{n-1}'{X}_{t+1} + {\mu}_{t+1}^{(n-1)} - {A}_{n} - {B}_{n}'{X}_{t} - {\mu}_{t}^{(n)} + {A}_{1} + {B}_{1}'{X}_{t} + {\mu}_{t}^{(1)}$

dado que ${X}_{t+1} = {\mu} + {\Phi}{X}_{t} + {\nu}_{t+1}$ temos que:

${rx}_{t+1}^{(n-1)} = {A}_{n-1} + {B}_{n-1}'({\mu} + {\Phi}{X}_{t} + {\nu}_{t+1}) + {\mu}_{t+1}^{(n-1)} - {A}_{n} - {B}_{n}'{X}_{t} - {\mu}_{t}^{(n)} + {A}_{1} + {B}_{1}'{X}_{t} + {\mu}_{t}^{(1)}$

também definimos anteriormente a fórmula de excesso de retorno como:

${rx}_{t+1}^{(n-1)} = {\beta}^{(n-1)'}({\lambda}_{0} + {\lambda}_{1}{X}_{t} + {\nu}_{t+1}) -\frac{1}{2}({\beta}^{(n-1)'}{\Sigma}{\beta}^{(n-1)} + {\sigma}^{2}) + {e}_{t+1}^{(n-1)}$

Igualando as duas fórmulas acima, assumindo que ${A}_{1} = -{\delta}_{0}$ e ${B}_{1} = -{\delta}_{1}$ e identificando os termos correspondentes temos que:

${A}_{n} = {A}_{n-1} + {B}_{n-1}'({\mu - {\lambda}_{0}}) + \frac{1}{2}({B}_{n-1}'{\Sigma}{B}_{n-1} + {\sigma}^{2}) - {\delta}_{0}$

${B}_{n}' = {B}_{n-1}'({\Phi} - {\lambda}_{1}) - {\delta}_{1}'$

${A}_{0} = 0$, ${B}_{0}' = 0$ 

e

${\beta}^{(n)} = {B}_{n}'$

também obtemos a seguinte equação para os erros de apreçamento:

${\mu}_{t+1}^{(n-1)} - {\mu}_{t}^{(n)} + {\mu}_{t}^{(1)} = {e}_{t+1}^{(n-1)}$

### Estimando a curva nominal implícita pelo modelo

${A}_{1}$ e ${B}_{1}$ são os fatores que definem o primeiro vertice da nossa curva de juros, ou seja, nossa taxa livre de risco. Para chegar nos valores de ${\delta}_{0}$ e ${\delta}_{1}$ fazemos um regressão dos PCs em relação a taxa livre de risco sendo ${\delta}_{0}$ a constante da nossa regressão e ${\delta}_{1}$ os betas.

In [ ]:
# Step 4 - Equation for the Short Rate

# Equation 21 of ACM calculating A_1 and B_1 when n=1
X_star = pca_factors.copy()
X_star["const"] = 1
X_star = X_star[["const"] + ["PC" + str(x) for x in range(1, n_factors + 1)]].values

# adjusting the risk free rate to the intended frequency
r_one = np.dot(1 / base_count, curve_exp.iloc[:, 0].values.T)

# Estimating A_1 and B_1
Delta_hat = (np.linalg.inv(X_star.T @ X_star) @ X_star.T) @ r_one
# A_1
d0_hat = Delta_hat[0]
# B_1
d1_hat = Delta_hat[1 : n_factors + 1]

In [ ]:
# Step 5 - Affine Recursions
def affine_recursions(
    Mu_hat: npt.NDArray[np.float64],
    Phi_hat: npt.NDArray[np.float64],
    Sigma_hat: npt.NDArray[np.float64],
    sigma2_hat: npt.NDArray[np.float64],
    lambda_0_hat: npt.NDArray[np.float64],
    lambda_1_hat: npt.NDArray[np.float64],
    delta_0_hat: npt.NDArray[np.float64],
    delta_1_hat: npt.NDArray[np.float64],
    pca_factors: pd.DataFrame,
    pca_columns: list[str],
    n_factors: int,
    tenors: int,
    sample_size: int,
    base_count: int,
) -> npt.NDArray[np.float64]:
    # our Xs are the PCAs
    X_star = pca_factors.copy()
    # Adding the constant on the Dataframe
    X_star["const"] = 1
    # Making sure the constant is on the first column
    X_star = X_star[["const"] + pca_columns].values

    N_rec = tenors.max()

    Bn = np.matrix(np.zeros((n_factors, N_rec + 1)))
    # as per ACM B_1 = - delta_1_hat
    Bn[:, 1] = -delta_1_hat.reshape((n_factors, 1))

    An = np.matrix(np.zeros((1, N_rec + 1)))
    # as per ACM A_1 = - delta_0_hat
    An[:, 1] = -delta_0_hat

    t_factor = np.matrix(np.zeros((sample_size, N_rec + 1)))
    t_factor[:, 1] = base_count

    for i in range(2, N_rec + 1):
        Bn[:, i] = ((Bn[:, i - 1].T @ (Phi_hat - lambda_1_hat)) - delta_1_hat.T).T
        An[:, i] = (
            An[:, i - 1]
            + Bn[:, i - 1].T @ (Mu_hat - lambda_0_hat)
            + 0.5 * (((Bn[:, i - 1].T @ Sigma_hat) @ Bn[:, i - 1]) + sigma2_hat)
            - delta_0_hat
        )
        t_factor[:, i] = base_count / i

    An = np.repeat(An, sample_size, axis=0)

    Xt = X_star[:, 1 : n_factors + 1].T

    # calculating yields, remember that they are continuous compounded rates
    return -np.multiply(t_factor, (An + (Bn.T @ Xt).T))

In [ ]:
# estimating the model implied curve
model_index = pca_factors[1:].index

model_implied_curve = affine_recursions(
    Mu_hat,
    Phi_hat,
    S_hat,
    sigma2_hat,
    l_zero_hat,
    l_one_hat,
    d0_hat,
    d1_hat,
    pca_factors[1:],
    pca_columns,
    n_factors,
    tenors,
    sample_size,
    base_count,
)

model_implied_curve = pd.DataFrame(
    data=model_implied_curve[:, 1:],
    index=model_index,
    columns=list(range(1, tenors.max() + 1)),
)

model_implied_curve = np.exp(model_implied_curve) - 1

Estimada a curva implícita pelo modelo, vamos comparar com a curva original para garantir que a estimação foi feita corretamente. 

Primeiro comparamos gráficamente as curvas nominais no último dia da amostra.

In [ ]:
nominal_curve = np.exp(curve_exp.loc[model_index, :]) - 1
nominal_curves = pd.DataFrame(
    columns=["Model Implied Curve", "Original Curve"],
    index=curve_exp.columns,
)
date_index = curve_exp.index.max()
nominal_curves.loc[:, "Model Implied Curve"] = model_implied_curve.loc[
    date_index, :
].values
nominal_curves.loc[:, "Original Curve"] = nominal_curve.loc[date_index, :].values
nominal_curves.index = nominal_curves.index / base_count

fig6 = plot_interest_rate_curve(
    nominal_curves,
    title=f"DI1 Curve and Model Implied Curve on {date_index.date()}",
    y_dtick=None,
)
fig6.show()


Abaixo calculamos a diferença entres as curvas no último dia da amostra

In [ ]:
model_difference = (model_implied_curve - nominal_curve).dropna(how="all")
model_error_on_date = pd.DataFrame(columns=["Model Error"], index=nominal_curve.columns)
model_error_on_date.loc[:, "Model Error"] = model_difference.loc[date_index, :].values
model_error_on_date.index = model_error_on_date.index / 12

fig7 = plot_interest_rate_curve(
    model_error_on_date * 10000,
    title=f"DI1 Model Error on {date_index.date()}",
    y_dtick=1,
    y_tickformat="0.1f",
    use_area_chart=True,
    yaxis_title="Model Error (bps)",
)
fig7.show()

Finalmente como um último teste, escolhemos um dos vértices da curva e comparamos as estimativas no tempo.

In [ ]:
n_months = 24
plot_series_nominal = pd.DataFrame(
    columns=["Nominal Curve", "Model Implied Curve"], index=model_index
)
plot_series_nominal.loc[:, "Nominal Curve"] = nominal_curve.loc[
    model_index, n_months - 1
].values
plot_series_nominal.loc[:, "Model Implied Curve"] = model_implied_curve.loc[
    model_index, n_months - 1
].values

fig9 = plot_interest_rate_on_time(
    plot_series_nominal, title=f"{str(int(n_months / 12))} Year Yield", y_dtick=None
)
fig9.show()


E novamente calculamos a diferença entre o modelo e a curva de juros original.

In [ ]:
this_nominal_curve = plot_series_nominal.loc[:, "Nominal Curve"]
this_model_implied_curve = plot_series_nominal.loc[:, "Model Implied Curve"]
plot_series_error = this_model_implied_curve - this_nominal_curve
plot_series_error.name = "Yield Error"
fig9_error = plot_interest_rate_on_time(
    plot_series_error,
    title=f"{str(int(n_months / 12))} Year Yield Error",
    y_dtick=None,
    use_area_chart=True,
)
fig9_error.show()

### Estimando a curva neutra ao risco

A partir do momento que temos todos os estimadores para a curva de juros do nosso modelo, é trivial calcularmos a curva de juros neutra ao risco, somente precisamos definir que $\hat{\lambda}_{0}$ e $\hat{\lambda}_{1}$ sejam iguais a zero.

In [ ]:
rny = affine_recursions(
    Mu_hat,
    Phi_hat,
    S_hat,
    sigma2_hat,
    0,
    0,
    d0_hat,
    d1_hat,
    pca_factors[1:],
    pca_columns,
    n_factors,
    tenors,
    sample_size,
    base_count,
)

model_index = pca_factors[1:].index
rny = pd.DataFrame(
    data=rny[:, 1:], index=model_index, columns=list(range(1, tenors.max() + 1))
)

rny = np.exp(rny) - 1

Calculada a curva neutra ao risco, podemos compará-la com a curva estimada para os preços de mercado.

In [ ]:
date_index = pd.to_datetime("2021-04-30")
date_index = monthly_curve.index[-1]
model_term_premium = (model_implied_curve - rny).dropna(how="all")
mic_on_date_curve = pd.DataFrame(
    columns=["Model Implied Curve", "Risk Neutral Curve"],
    index=model_implied_curve.columns,
)
max_year = None
mic_on_date_curve.loc[:, "Model Implied Curve"] = model_implied_curve.loc[
    date_index, :
].values
mic_on_date_curve.loc[:, "Risk Neutral Curve"] = rny.loc[date_index, :].values
mic_on_date_curve.index = mic_on_date_curve.index / 12
if max_year is not None:
    mic_plot_curve = mic_on_date_curve[mic_on_date_curve.index <= max_year]
else:
    mic_plot_curve = mic_on_date_curve

fig8 = plot_interest_rate_curve(
    mic_plot_curve,
    title=f"DI1 Curve Term Risk Premium on {date_index.date()}",
    y_dtick=None,
)
fig8.show()

Agora comparamos a curva implícita pelo modelo com a curva neutra ao risco ao longo do tempo para um único vertice

In [ ]:
n_months2 = 12
plot_series2 = pd.DataFrame(
    columns=["Nominal Curve", "Risk Neutral Curve"], index=model_index
)
plot_series2.loc[:, "Nominal Curve"] = nominal_curve.iloc[:, n_months2 - 1].values
plot_series2.loc[:, "Risk Neutral Curve"] = rny.iloc[:, n_months2 - 1].values

fig4 = plot_interest_rate_on_time(
    plot_series2, title=f"{str(int(n_months2 / 12))} Year Yield", y_dtick=None
)
fig4.show()

In [ ]:
n_months3 = 120
plot_series3 = pd.DataFrame(
    columns=["Nominal Curve", "Risk Neutral Curve"], index=model_index
)
plot_series3.loc[:, "Nominal Curve"] = nominal_curve.iloc[:, n_months3 - 1].values
plot_series3.loc[:, "Risk Neutral Curve"] = rny.iloc[:, n_months3 - 1].values

fig4 = plot_interest_rate_on_time(
    plot_series3,
    title=f"{str(int(n_months3 / 12))} Year Yield",
    y_dtick=None,
    y_tickformat=".2f",
)
fig4.show()

Por último calculamos somente o prêmio de risco do modelo para os vértices escolhidos no tempo

In [ ]:
term_premium_n_months1 = 24
term_premium_name1 = f"{str(int(term_premium_n_months1 / 12))} Year"
term_premium_n_months2 = 60
term_premium_name2 = f"{str(int(term_premium_n_months2 / 12))} Years"

term_premium = ((1 + nominal_curve) / (1 + rny) - 1).dropna(how="all")

term_plot_series = pd.DataFrame(
    columns=[term_premium_name1, term_premium_name2], index=term_premium.index
)
term_plot_series.loc[:, term_premium_name1] = term_premium.iloc[
    :, term_premium_n_months1 - 1
].values
term_plot_series.loc[:, term_premium_name2] = term_premium.iloc[
    :, term_premium_n_months2 - 1
].values

fig5 = plot_interest_rate_on_time(term_plot_series, title="Yield Premium", y_dtick=None)
fig5.show()

In [ ]:
n_months3 = 120

plot_series4 = pd.DataFrame(
    columns=["Nominal Curve", "Term Premium"], index=model_index
)
plot_series4.loc[:, "Nominal Curve"] = nominal_curve.iloc[:, n_months3 - 1].values
plot_series4.loc[:, "Term Premium"] = term_premium.iloc[:, n_months3 - 1].values

fig6 = plot_interest_rate_on_time(
    plot_series4, title=f"{str(int(n_months3 / 12))} Year Yield", y_dtick=None
)
fig6.show()